# ChemCharts JSON Plot 

what is my aim?
what can I do?
how to install me?

In [1]:
# load dependencies

import os
import json
import tempfile

# change me

chemcharts_dir = os.path.expanduser("~/Documents/Projects/ChemCharts") # set project folder on your device
chemcharts_env = os.path.expanduser("~/miniconda3/envs/chemcharts")    # set environment path on your device
output_dir = os.path.expanduser("~/Desktop/chemcharts_json_test")      # set output path on your device

# do NOT change me

try: ipynb_path                                                   
except NameError: ipynb_path = os.getcwd()                             # sets root path

try:
    os.mkdir(output_dir)                                               # checks whether there is an output folder 
except FileExistsError:                                                # and generates one if there isn't one already available
    pass


In [2]:
# initialize configuration dictionary

conf={"chemcharts": {}}
    

In [3]:
# add header

conf={
  "chemcharts": {
    "header": {}                                                        # leave empty if not needed
  }
}


In [4]:
# with the execution step you can add the desired tasks, start with data loading

conf["chemcharts"]["execution"] = [
      {"task":  "data_loading",
       "input":  os.path.join(chemcharts_dir, "data/scaffold_memory.csv"),
       "input_type":  "csv",
       "columns":  {
                     "smiles_column": "SMILES",
                     "scores_column": "total_score",
                     "epochs_column": "Step"
                    }
      }]
    

In [5]:
# then generate RDKIT fingerprints 

conf["chemcharts"]["execution"].append(
      {"task":  "generate_fingerprints",
       "type":  "maccs",                                                # you can choose between standard, morgan and maccs
       "parameters":  {"useFeatures": True}                             # with the other two fingerprints there are no parameters
      })


In [6]:
# reduce the dimensionality with UMP

conf["chemcharts"]["execution"].append(
      {"task":  "dimensional_reduction",
       "type": "UMAP",
       "parameters":  {}
      })


In [7]:
# optional: filter a given range of data

conf["chemcharts"]["execution"].append(
      {"task":  "filtering_data",
       "type": "filtering",
       "parameters":  {"range_dim1": [-100, 100],                       # set range for UMAP_1
                       "range_dim2": [-100, 100]}                       # set range for UMAP_2
      })


In [8]:
# optional: cluster the data with KMeans

conf["chemcharts"]["execution"].append(
      {"task":  "clustering_data",
       "type": "KMmeans",
       "parameters":  {"k": 10}                                         # set the number of desired KMeans clusters
      })


In [9]:
# optional: bin the scores and return their median

conf["chemcharts"]["execution"].append(
      {"task":  "binning_scores",
       "type": "binning",
       "parameters":  {"num_bins": 4}                                    # set the number of desired bins
      })


In [13]:
# write out the result and use the pkl file from now onwards

conf["chemcharts"]["execution"].append({
        "task": "write_out",
        "format": "pkl",
        "path": os.path.join(output_dir, "simple_plot_test.pkl")              # choose loaction for ".pkl" file
      })


In [14]:
# now you can start generating plots

conf={
  "chemcharts": {
    "header": {},
    "execution": [
      {"task":  "data_loading",
       "input":  ["simple_plot_test.pkl"],
       "input_type":  "pkl"
      },
      {"task": "generate_plot",
       "type": "hexagonal_plot",                                             # choose plot type (here: Hexagonal)
       "parameters":  {"title": "Hexagonal ChemCharts Plot",                 # set plot title (default: Hexagonal Chemcharts Plot)
                       "gridsize": 20,                                       # set gridsize {default: 20}
                       "fontsize": 14,                                       # set fontsize of title (default: 14) 
                       "top": 0.9,                                           # set top margin (default: 0.9)    
                       "color": "#4CB391"},                                  # set marker color (default: #4CB391)
        "settings": {"path": os.path.join(output_dir, "hexagonal_plot.png"), # set output path including file name
                     "format": "png",                                        # set file format (default: png)  
                     "dpi": 150}                                             # set dpi (default: 150)
      }
    ]
  }
}


# write-out of configuration file

configuration_JSON_path = os.path.join(output_dir, "chemcharts_config_plots.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(conf, f, indent=4, sort_keys=True)
    
    
#execution

!{chemcharts_env}/bin/python {chemcharts_dir}/chemcharts_json.py -conf {configuration_JSON_path}

Traceback (most recent call last):
  File "/home/nutzer/Documents/Projects/ChemCharts/chemcharts_json.py", line 101, in <module>
    with open(task[_JE.INPUT][0], "rb") as dill_file:
FileNotFoundError: [Errno 2] No such file or directory: 'simple_plot_test.pkl'


In [ ]:
# you can set xlim and ylim to define an excerpt

conf={
  "chemcharts": {
    "header": {},
    "execution": [
      {"task":  "data_loading",
       "input":  ["simple_plot_test.pkl"],
       "input_type":  "pkl"
      },
      {"task": "generate_plot",
       "type": "hexagonal_plot",                                             
       "parameters":  {"title": "Hexagonal ChemCharts Plot",                 
                       "gridsize": 20,                                       
                       "fontsize": 14,                                        
                       "top": 0.9,                                           
                       "xlim": [-6.5, 14],                                   # set xlim
                       "ylim": [-9, 12],                                     # set ylim   
                       "color": "#4CB391"},                                  
        "settings": {"path": os.path.join(output_dir, "hexagonal_plot.png"), 
                     "format": "png",                                          
                     "dpi": 150}                                             
      }
    ]
  }
}


# write-out of configuration file

configuration_JSON_path = os.path.join(output_dir, "chemcharts_config_plots.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(conf, f, indent=4, sort_keys=True)
    

#execution

!{chemcharts_env}/bin/python {chemcharts_dir}/chemcharts_json.py -conf {configuration_JSON_path}


In [ ]:
# you can also add multiple plot generations

conf={
  "chemcharts": {
    "header": {},
    "execution": [
      {"task":  "data_loading",
       "input":  os.path.join(chemcharts_dir, "data/scaffold_memory.csv"),
       "input_type":  "csv",
       "columns":  {
                     "smiles_column": "SMILES",
                     "scores_column": "total_score",
                     "epochs_column": "Step"
                    }
      },
      {"task":  "generate_fingerprints",
       "type":  "maccs",
       "parameters":  {"useFeatures": True}
      },
      {"task":  "dimensional_reduction",
       "type": "UMAP",
       "parameters":  {}
      },
      {"task":  "filtering_data",
       "type": "filtering",
       "parameters":  {"range_dim1": [-100, 100],
                       "range_dim2": [-100, 100]}
      },
      {"task":  "clustering_data",
       "type": "KMmeans",
       "parameters":  {"k": 10}
      },
      {"task":  "binning_scores",
       "type": "binning",
       "parameters":  {"num_bins": 4}
      },
      {
        "task": "write_out",
        "format": "pkl",
        "path": os.path.join(output_dir, "simple_plot_test.pkl")
      },
      {"task": "generate_plot",
       "type": "hexagonal_plot",
       "parameters":  {"title": "Hexagonal ChemCharts Plot",
                       "gridsize": 20,
                       "fontsize": 14,
                       "top": 0.9,
                       "color": "#4CB391"},
        "settings": {"path": os.path.join(output_dir, "hexagonal_plot.png"),
                     "format": "png",
                     "dpi": 150}
      },
      {"task": "generate_plot",
       "type": "histogram_plot",                                              # choose plot type (here: Histogram)
       "parameters":  {"title": "Histogram ChemCharts Plot",                  # set plot title (default: Histogram Chemcharts Plot)
                       "bins": 20,                                            # set bin number {default: 20}
                       "fontsize": 14,                                        # set fontsize of title (default: 14) 
                       "top": 0.9,                                            # set top margin (default: 0.9)
                       "color": "#d11d80"},                                   # set marker color (default: #d11d80)
       "settings": {"path": os.path.join(output_dir, "histogram_plot.png"),   # set output path including file name
                    "format": "png",                                          # set file format (default: png)  
                    "dpi": 300,                                               # set dpi (default: 300)
                    "figsize": [17, 17]}                                      # set figuresize (default: [17, 17])
      }
    ]
  }
}


In [ ]:
# other plot variants ...
 
    # Scatter Boxplot

     {"task": "generate_plot",
       "type": "scatter_boxplot_plot",
       "parameters":  {"title": "Scatter Boxplot ChemCharts Plot",                   # set plot title (default: Scatter Boxplot Chemcharts Plot)
                       "fontsize": 14,                                               # set fontsize of title (default: 14) 
                       "top": 0.9},                                                  # set top margin (default: 0.9)
        "settings": {"path": os.path.join(output_dir, "scatter_boxplot_plot.png"),   # set output path including file name
                     "format": "png",                                                # set file format (default: png) 
                     "dpi": 150,                                                     # set dpi (default: 150)
                     "figsize": [17, 17]}                                            # set figuresize (default: [17, 17])
      }
    
    
    # Scatter Interactive
    
      {"task": "generate_plot",
       "type": "scatter_interactive_plot",
       "parameters":  {"title": "Scatter Interactive ChemCharts Plot",                # set plot title (default: Scatter Interactive Chemcharts Plot)
                       "marker_size": 1},                                             # set marker size (default: 1)
       "settings": {"path": os.path.join(output_dir, "scatter_interactive_plot.png"), # set output path including file name
                    "view": False,                                                    # set view on true if pop-up window is desired (default: false)
                     "format": "png"}                                                 # set file format (default: png) 
      }
        
    # Scatter Static
    
      {"task": "generate_plot",
       "type": "scatter_static_plot",
       "parameters":  {"title": "Scatter Static ChemCharts Plot",                     # set plot title (default: Scatter Static Chemcharts Plot)
                       "color": "#0000ff",                                            # set marker color (default: #0000ff)
                       "s": 6},                                                       # set marker size (default: 6)
        "settings": {"path": os.path.join(output_dir, "scatter_static_plot.png"),     # set output path including file name
                     "format": "png",                                                 # set file format (default: png) 
                     "dpi": 150,                                                      # set dpi (default: 150)
                     "figsize": [17, 17]}                                             # set figuresize (default: [17, 17])
      }
    
    
    # Trisurf Interactive
    
      {"task": "generate_plot",
       "type": "trisurf_interactive_plot",
       "parameters":  {"title": "Trisurf Interactive ChemCharts Plot",                 # set plot title (default: Trisurf Interactive Chemcharts Plot)
                       "color": "Portland"},                                           # set plotly built-in continuous color scales (default: Portland)
       "settings": {"path": os.path.join(output_dir, "trisurf_interactive_plot.png"),  # set output path including file name
                    "view": True,                                                      # set view on true if pop-up window is desired (default: false)
                     "format": "png"}                                                  # set file format (default: png) 
      }
       
        
    # Trisurf Static
    
      {"task": "generate_plot",
       "type": "trisurf_static_plot",
       "parameters":  {"title": "Trisurf Static ChemCharts Plot",                       # set plot title (default: Trisurf Static Chemcharts Plot)
                       "color": "gist_rainbow"},                                        # set matplotlib colormap (default: gist_rainbow)
        "settings": {"path": os.path.join(output_dir, "trisurf_static_plot.png"),       # set output path including file name
                     "format": "png",                                                   # set file format (default: png) 
                     "dpi": 150,                                                        # set dpi (default: 150)
                     "figsize": [9, 9]}                                                 # set figuresize (default: [9, 9])
      }
    

In [ ]:
# write-out of configuration file

configuration_JSON_path = os.path.join(output_dir, "chemcharts_config_plots.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(conf, f, indent=4, sort_keys=True)
    

In [ ]:
# progress statements 

%%capture captured_std_stream --no-stdout
%%capture captured_err_stream --no-stderr


In [ ]:
# execution

!{chemcharts_env}/bin/python {chemcharts_dir}/chemcharts_json.py -conf {configuration_JSON_path}
